### SparklyRGT Template: Baseline and Acquisition Analysis 

**Requirements**
* The data must be an excel file from MEDPC2XL (trial by trial data) 
* The data, sparklyRGT.py file, and this notebook must all be in the same folder

**Getting started: Please make a copy of this (sparklyRGT_template_2) for each analysis**
- Refer to sparklyRGT_documentation for function information
- Note: depending on your analysis, you will only have to complete certain sections of the sparklyRGT_documentation
- Note: feel free to create a personal template once you've become comfortable - this is just an example

In [1]:
import os
os.chdir('..')
import sparklyRGT as rgt 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import scipy.stats as stats
pd.options.mode.chained_assignment = None
pd.set_option('display.max_rows',100)

I am being executed!


***

# 1) Load data into Python



In [3]:
#remove the leading 'M' from the TF file subject numbers and convert to integer/float 

In [4]:
#add header names from BH03 to other files so they can be concatenated properly 

In [2]:
from os import listdir
#data needs to be loaded in from OSF
#either download them from OSF and upload to github, or load in directly from OSF 

#OSF files are CSVs (except BH03 is xlsx) and load_multiple_data loads in excel files
#sparklyrgt.py needs to be edited so that either excel files or CSVs can be loaded in

path = '../sparklyRGT_tutorial/data/'
file_names = [f for f in listdir(path)]

df = rgt.load_multiple_data(file_names, path, reset_sessions = False)

df.head()

,MSN,StartDate,StartTime,Subject,Group,Box,Experiment,Comment,Session,Trial,...,Pun_Persev_H5,Pun_HeadEntry,Pun_Dur,Premature_Resp,Premature_Hole,Rew_Persev_H1,Rew_Persev_H2,Rew_Persev_H3,Rew_Persev_H4,Rew_Persev_H5
0,rGT_A-cue,01/23/16,8:13:19,173,0.0,1,0.0,NaN,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,rGT_A-cue,01/23/16,8:13:19,173,0.0,1,0.0,NaN,1.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,rGT_A-cue,01/23/16,8:13:19,173,0.0,1,0.0,NaN,1.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,rGT_A-cue,01/23/16,8:13:19,173,0.0,1,0.0,NaN,1.0,4.0,...,2.0,1.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,rGT_A-cue,01/23/16,8:13:19,173,0.0,1,0.0,NaN,1.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Data cleaning

### Check session numbers for each rat and drop subjects

In [16]:
#missing data

#we need to only take rats that have consecutive sessions from 1 to 5
#some rats will be missing sessions and need to be excluded

# removes subjects where the first 5 sessions are not consecutive
subjects = df.drop_duplicates(subset=['Subject', 'Session'])
subjects_n = subjects[['Subject', 'Session']]
subjects_n = subjects_n.sort_values(by=['Subject', 'Session'])

n = subjects_n['Subject'].nunique()
drop_subs = []

i = 0
temp = subjects_n
while i < n:
    check_consec = temp.head()
    num = check_consec['Subject'].iloc[0]
    con_list = check_consec['Session'].tolist()
    con_list = list(map(int, con_list))
    if sorted(con_list) != list(range(min(con_list), max(con_list)+1)):
        drop_subs.append(num)
    i = i + 1
    temp = temp[temp.Subject != num]

#to drop subjects:
df2 = rgt.drop_subjects(df, drop_subs)

In [25]:
#check if there are any subjects that were run on more than one task version (including 5CSRT, FC)

#to display all the task names in the dataframe:
df2.MSN.unique()

tasks = df2.drop_duplicates(subset=['Subject', 'MSN'])
tasks_n = tasks[['Subject', 'MSN']]
tasks_n = tasks_n.sort_values(by=['Subject'])

tasks_n_dup = tasks_n[tasks_n.duplicated(['Subject'], keep=False)]

for index, row in tasks_n.iterrows():
    if row['Subject'] in tasks_n_dup['Subject'].values:
        tasks_n.drop(index, inplace=True)

#drop any subjects that were run on more than one task 

        Subject              MSN
245447      708  rGT_FC_ClassicB
257421      708     rGT_ClassicB
259323      711     RGT_ClassicA
244323      711  RGT_FC_ClassicA
244670      715  RGT_FC_ClassicB
250334      715     RGT_ClassicB
246709      724  rGT_FC_ClassicB
258986      724     rGT_ClassicB
264652      727     RGT_ClassicA
245735      727  RGT_FC_ClassicA
246050      731  RGT_FC_ClassicB
255018      731     RGT_ClassicB
252178      732     RGT_ClassicB
246106      732  RGT_FC_ClassicB
502623     1404        rGT_A-cue
499246     1404    rGT_A-cue-FC2
513392     1407        rGT_B-cue
499497     1407     rGT_B-cue-FC
499568     1408     rGT_B-cue-FC
510075     1408        rGT_B-cue
501477     1412        rGT_A-cue
495837     1412    rGT_A-cue-FC2
496430     1419        rGT_A-cue
500362     1419        rGT_B-cue
496520     1420        rGT_B-cue
500463     1420     RGT_ClassicA
496778     1424        rGT_B-cue
500682     1424     RGT_ClassicB
500773     1425     RGT_ClassicB
496887    

In [ ]:
# split the dataframe by cued and classic rats, and save them as two separate CSVs (both with column headers)

#all cued tasks will have 'cue' in the MSN (A and B version)
#all classic tasks should have 'Classic' (A and B) - either rGT or RGT 

#upload to sparklyRGT/data 